In [1]:
!pip install autogluon --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracki

In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 36.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg

import pandas as pd
import numpy as np
import glob
import pickle
import os
from typing import Sequence, Literal
from pathlib import Path
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [ ]:
def smiles_to_bits(smiles):
    N_BITS, RADIUS = 2048, 2
    gen = rfg.GetMorganGenerator(radius=RADIUS, fpSize=N_BITS)

    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.full(N_BITS, np.nan, dtype="float32")
    fp = gen.GetFingerprint(mol)
    arr = np.zeros(N_BITS, dtype="uint8")
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr


In [ ]:
df = (
    pd.read_csv("/content/drive/MyDrive/04_tech/molecule_embeddings/freesolv.csv")
    .rename(
        columns={
            "# compound id (and file prefix)": "id",
            " SMILES": "smiles",
            " experimental value (kcal/mol)": "dG",
        }
    )
    .loc[:, ["id", "smiles", "dG"]]
)

bit_mat = np.vstack(
    df["smiles"].map(smiles_to_bits).to_numpy()
)  # shape = (n_samples, 2048)

bit_cols = [f"morgan_{i}" for i in range(bit_mat.shape[1])]
bits_df = pd.DataFrame(bit_mat, columns=bit_cols, dtype="uint8")

df = pd.concat([df.reset_index(drop=True), bits_df], axis=1, copy=False)


In [ ]:
drop_cols = ["id", "smiles"]
data = df.drop(columns=drop_cols, errors="ignore")


train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

predictor = TabularPredictor(label="dG", eval_metric="root_mean_squared_error").fit(
    train_data=train_df,
    # presets="best_quality",
    presets="medium_quality",
    time_limit=None,
)

perf = predictor.evaluate(test_df)  # RMSE, MAE などが表示される
print(perf)

pred = predictor.predict(test_df.drop(columns=["dG"]))
print("予測値 (kcal/mol) 上位 5 行:\n", pred.head())


No path specified. Models will be saved in: "AutogluonModels/ag-20250517_011739"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       9.96 GB / 12.67 GB (78.6%)
Disk Space Avail:   64.79 GB / 107.72 GB (60.1%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "/content/AutogluonModels/ag-20250517_011739"
Train Data Rows:    513
Train Data Columns: 2048
Label Column:       dG
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (3.43, -23.62, -3.91591, 3.80544)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during P

[1000]	valid_set's rmse: 2.35152


	-2.3362	 = Validation score   (-root_mean_squared_error)
	7.46s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 2.35152


	-2.3362	 = Validation score   (-root_mean_squared_error)
	1.16s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-2.5728	 = Validation score   (-root_mean_squared_error)
	11.88s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-2.1533	 = Validation score   (-root_mean_squared_error)
	126.52s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-2.561	 = Validation score   (-root_mean_squared_error)
	7.42s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-2.5583	 = Validation score   (-root_mean_squared_error)
	2.58s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-2.3719	 = Validation score   (-root_mean_squared_error)
	4.67s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-2.0566	 = Validation score   (-root_mean_squared_error)
	87.84s	 = Training   runtime
	0.01s	 = Validation ru

{'root_mean_squared_error': np.float64(-1.7978520320349516), 'mean_squared_error': -3.2322719290922044, 'mean_absolute_error': -1.1703882541501707, 'r2': 0.7959508073297211, 'pearsonr': 0.9131335523567485, 'median_absolute_error': np.float64(-0.7264052486419679)}
予測値 (kcal/mol) 上位 5 行:
 86     -3.204861
192    -5.080924
479   -10.767002
616   -14.403172
72     -0.634033
Name: dG, dtype: float32


In [ ]:
print(predictor.evaluate(test_df))

{'root_mean_squared_error': np.float64(-1.7978520320349516), 'mean_squared_error': -3.2322719290922044, 'mean_absolute_error': -1.1703882541501707, 'r2': 0.7959508073297211, 'pearsonr': 0.9131335523567485, 'median_absolute_error': np.float64(-0.7264052486419679)}


In [ ]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2.007012,root_mean_squared_error,0.102625,221.851143,0.000822,0.030184,2,True,10
1,NeuralNetTorch,-2.056607,root_mean_squared_error,0.012953,87.843807,0.012953,87.843807,1,True,8
2,CatBoost,-2.153327,root_mean_squared_error,0.074074,126.517733,0.074074,126.517733,1,True,4
3,LightGBM,-2.336219,root_mean_squared_error,0.012866,1.163880,0.012866,1.163880,1,True,2
4,LightGBMXT,-2.336219,root_mean_squared_error,0.014776,7.459419,0.014776,7.459419,1,True,1
5,LightGBMLarge,-2.352803,root_mean_squared_error,0.030232,4.778205,0.030232,4.778205,1,True,9
6,XGBoost,-2.371899,root_mean_squared_error,0.010430,4.669916,0.010430,4.669916,1,True,7
7,NeuralNetFastAI,-2.558344,root_mean_squared_error,0.019741,2.583475,0.019741,2.583475,1,True,6
8,ExtraTreesMSE,-2.561037,root_mean_squared_error,0.105080,7.424413,0.105080,7.424413,1,True,5
9,RandomForestMSE,-2.572756,root_mean_squared_error,0.102849,11.877224,0.102849,11.877224,1,True,3


In [ ]:
def _load_vec(path: Path, model: str) -> np.ndarray:
    with path.open("rb") as f:
        emb_dict = pickle.load(f)
    vec = emb_dict["emb"][model]
    if hasattr(vec, "detach"):  # torch.Tensor の場合
        vec = vec.detach().cpu().numpy()
    return np.asarray(vec, dtype="float32").flatten()


def calc_nnp_emb_merge(
    df: pd.DataFrame,
    model: str,
    emb_dir="/content/drive/MyDrive/04_tech/molecule_embeddings/pyg/embeddings",
    id_dtype: Literal["str", "int"] = "str",  # id の型を合わせるため
) -> pd.DataFrame:
    emb_paths = sorted(Path(emb_dir).glob("*.pkl"))
    if not emb_paths:
        raise FileNotFoundError(f"No *.pkl found in {emb_dir}")

    first_vec = _load_vec(emb_paths[0], model)
    dim = first_vec.size
    emb_cols = [f"{model}_{i}" for i in range(dim)]

    ids, vecs = [], []
    for p in emb_paths:
        vec = _load_vec(p, model)
        if vec.size != dim:
            continue
        ids.append(p.stem)
        vecs.append(vec)

    emb_mat = np.vstack(vecs)
    emb_df = pd.DataFrame(emb_mat, columns=emb_cols, dtype="float32")
    emb_df.insert(0, "id", ids)

    if id_dtype == "int":
        emb_df["id"] = emb_df["id"].astype(int)
    else:
        df["id"] = df["id"].astype(str)
        emb_df["id"] = emb_df["id"].astype(str)

    df_out = (
        df.reset_index(drop=False)
        .merge(emb_df, on="id", how="left", validate="one_to_one")
        .sort_values("index")
        .drop(columns="index")
        .reset_index(drop=True)
    )
    return df_out


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/04_tech/molecule_embeddings/freesolv_schnet.csv")
drop_cols = ["id", "smiles"]
data = df.drop(columns=drop_cols, errors="ignore")


train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

predictor = TabularPredictor(label="dG", eval_metric="root_mean_squared_error").fit(
    train_data=train_df,
    # presets="best_quality",
    presets="medium_quality",
    time_limit=None,
)

perf = predictor.evaluate(test_df)  # RMSE, MAE などが表示される
print(perf)

pred = predictor.predict(test_df.drop(columns=["dG"]))
print("予測値 (kcal/mol) 上位 5 行:\n", pred.head())


No path specified. Models will be saved in: "AutogluonModels/ag-20250517_014152"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       8.81 GB / 12.67 GB (69.5%)
Disk Space Avail:   64.46 GB / 107.72 GB (59.8%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "/content/AutogluonModels/ag-20250517_014152"
Train Data Rows:    513
Train Data Columns: 128
Label Column:       dG
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (3.43, -23.62, -3.91591, 3.80544)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Pr

[1000]	valid_set's rmse: 1.89377


	-1.885	 = Validation score   (-root_mean_squared_error)
	5.0s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 1.92974
[2000]	valid_set's rmse: 1.91613
[3000]	valid_set's rmse: 1.91546


	-1.9153	 = Validation score   (-root_mean_squared_error)
	27.73s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-2.1057	 = Validation score   (-root_mean_squared_error)
	8.29s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	-1.9055	 = Validation score   (-root_mean_squared_error)
	41.31s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-2.067	 = Validation score   (-root_mean_squared_error)
	2.83s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-2.3948	 = Validation score   (-root_mean_squared_error)
	1.92s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-1.9292	 = Validation score   (-root_mean_squared_error)
	12.43s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-1.9366	 = Validation score   (-root_mean_squared_error)
	48.48s	 = Training   runtime
	0.08s	 = Validation ru

{'root_mean_squared_error': np.float64(-1.9732910824870364), 'mean_squared_error': -3.8938776962228596, 'mean_absolute_error': -1.087744305274168, 'r2': 0.7541844814726866, 'pearsonr': 0.8822315391534217, 'median_absolute_error': np.float64(-0.5188237929344177)}
予測値 (kcal/mol) 上位 5 行:
 86     -0.659733
192    -4.775836
479   -11.183632
616   -11.739090
72      1.539239
Name: dG, dtype: float32


In [ ]:
print(predictor.evaluate(test_df))

{'root_mean_squared_error': np.float64(-1.9732910824870364), 'mean_squared_error': -3.8938776962228596, 'mean_absolute_error': -1.087744305274168, 'r2': 0.7541844814726866, 'pearsonr': 0.8822315391534217, 'median_absolute_error': np.float64(-0.5188237929344177)}


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/04_tech/molecule_embeddings/freesolv_molformer.csv")
drop_cols = ["id", "smiles"]
data = df.drop(columns=drop_cols, errors="ignore")


train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

predictor = TabularPredictor(label="dG", eval_metric="root_mean_squared_error").fit(
    train_data=train_df,
    # presets="best_quality",
    presets="medium_quality",
    time_limit=None,
)

perf = predictor.evaluate(test_df)  # RMSE, MAE などが表示される
print(perf)

pred = predictor.predict(test_df.drop(columns=["dG"]))
print("予測値 (kcal/mol) 上位 5 行:\n", pred.head())


No path specified. Models will be saved in: "AutogluonModels/ag-20250517_014646"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       9.02 GB / 12.67 GB (71.2%)
Disk Space Avail:   64.39 GB / 107.72 GB (59.8%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "/content/AutogluonModels/ag-20250517_014646"
Train Data Rows:    513
Train Data Columns: 768
Label Column:       dG
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (3.43, -23.62, -3.91591, 3.80544)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Pr

[1000]	valid_set's rmse: 2.25922
[2000]	valid_set's rmse: 2.25581
[3000]	valid_set's rmse: 2.25505
[4000]	valid_set's rmse: 2.25494


(_dystack pid=25005) 	-1.9857	 = Validation score   (-root_mean_squared_error)
(_dystack pid=25005) 	121.64s	 = Training   runtime
(_dystack pid=25005) 	0.52s	 = Validation runtime
(_dystack pid=25005) Fitting model: NeuralNetTorch_BAG_L1 ...
(_dystack pid=25005) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.02%)


(_ray_fit pid=33124) [1000]	valid_set's rmse: 3.44815
(_ray_fit pid=33124) [2000]	valid_set's rmse: 3.40594
[5000]	valid_set's rmse: 2.25491
(_ray_fit pid=33124) [3000]	valid_set's rmse: 3.38492
[6000]	valid_set's rmse: 2.25491
[7000]	valid_set's rmse: 2.25491
[8000]	valid_set's rmse: 2.25491
[9000]	valid_set's rmse: 2.25491
[10000]	valid_set's rmse: 2.25491


	-2.2549	 = Validation score   (-root_mean_squared_error)
	240.76s	 = Training   runtime
	0.36s	 = Validation runtime
Fitting model: LightGBM ...
	-2.4271	 = Validation score   (-root_mean_squared_error)
	39.31s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-2.5041	 = Validation score   (-root_mean_squared_error)
	68.99s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: CatBoost ...
(_dystack pid=25005) 	-1.939	 = Validation score   (-root_mean_squared_error)
(_dystack pid=25005) 	417.41s	 = Training   runtime
(_dystack pid=25005) 	0.23s	 = Validation runtime
(_dystack pid=25005) Fitting model: LightGBMLarge_BAG_L1 ...
(_dystack pid=25005) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=7.13%)


...

(_ray_fit pid=36265) [1000]	valid_set's rmse: 2.74324


(_dystack pid=25005) 	-2.2707	 = Validation score   (-root_mean_squared_error)
(_dystack pid=25005) 	134.27s	 = Training   runtime
(_dystack pid=25005) 	0.17s	 = Validation runtime
(_dystack pid=25005) Fitting model: CatBoost_r177_BAG_L1 ...
(_dystack pid=25005) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 12.81% memory usage per fold, 51.23%/80.00% total).
(_dystack pid=25005) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=12.81%)
(_dystack pid=25005) 	-1.9662	 = Validation score   (-root_mean_squared_error)
(_dystack pid=25005) 	935.66s	 = Training   runtime
(_dystack pid=25005) 	1.74s	 = Validation runtime
(_dystack pid=25005) Fitting model: NeuralNetTorch_r79_BAG_L1 ...
(_dystack pid=25005) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.03%)
(_dystack pid=4887) 	-1.8713	 = 

(_ray_fit pid=42772) [1000]	valid_set's rmse: 1.86599
(_ray_fit pid=42927) [1000]	valid_set's rmse: 1.76166
(_ray_fit pid=42927) [2000]	valid_set's rmse: 1.74796
(_ray_fit pid=43064) [1000]	valid_set's rmse: 1.64889
(_ray_fit pid=43064) [2000]	valid_set's rmse: 1.63125
(_ray_fit pid=43210) [1000]	valid_set's rmse: 2.54345
(_ray_fit pid=43210) [2000]	valid_set's rmse: 2.50989
(_ray_fit pid=43258) [3000]	valid_set's rmse: 2.97762 [repeated 3x across cluster]
(_ray_fit pid=43258) [6000]	valid_set's rmse: 2.92336 [repeated 3x across cluster]
(_ray_fit pid=43258) [8000]	valid_set's rmse: 2.89642 [repeated 2x across cluster]
(_ray_fit pid=43415) [1000]	valid_set's rmse: 1.63281 [repeated 3x across cluster]
(_ray_fit pid=43609) [1000]	valid_set's rmse: 2.64911
(_ray_fit pid=43609) [2000]	valid_set's rmse: 2.57333


(_dystack pid=25005) 	-2.1277	 = Validation score   (-root_mean_squared_error)
(_dystack pid=25005) 	161.35s	 = Training   runtime
(_dystack pid=25005) 	1.04s	 = Validation runtime
(_dystack pid=25005) Fitting model: NeuralNetFastAI_r191_BAG_L1 ...
(_dystack pid=25005) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.05%)
	-2.7603	 = Validation score   (-root_mean_squared_error)
	287.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.8, 'NeuralNetFastAI': 0.2}
	-1.789	 = Validation score   (-root_mean_squared_error)
	0.02s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2227.65s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 1723.4 rows/s (103 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-202

{'root_mean_squared_error': np.float64(-1.5268422502117474), 'mean_squared_error': -2.3312472570316722, 'mean_absolute_error': -1.0737895716380241, 'r2': 0.8528313424280135, 'pearsonr': 0.929452186212399, 'median_absolute_error': np.float64(-0.8002648162841801)}
予測値 (kcal/mol) 上位 5 行:
 86     -2.435574
192    -4.146752
479   -13.566082
616   -17.282131
72      0.919202
Name: dG, dtype: float32


In [ ]:
print(predictor.evaluate(test_df))

{'root_mean_squared_error': np.float64(-1.5268422502117474), 'mean_squared_error': -2.3312472570316722, 'mean_absolute_error': -1.0737895716380241, 'r2': 0.8528313424280135, 'pearsonr': 0.929452186212399, 'median_absolute_error': np.float64(-0.8002648162841801)}


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/04_tech/molecule_embeddings/freesolv_dimenet.csv")
drop_cols = ["id", "smiles"]
data = df.drop(columns=drop_cols, errors="ignore")


train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

predictor = TabularPredictor(label="dG", eval_metric="root_mean_squared_error").fit(
    train_data=train_df,
    # presets="best_quality",
    presets="medium_quality",
    time_limit=None,
)

perf = predictor.evaluate(test_df)  # RMSE, MAE などが表示される
print(perf)

pred = predictor.predict(test_df.drop(columns=["dG"]))
print("予測値 (kcal/mol) 上位 5 行:\n", pred.head())


No path specified. Models will be saved in: "AutogluonModels/ag-20250517_022734"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       8.59 GB / 12.67 GB (67.7%)
Disk Space Avail:   64.17 GB / 107.72 GB (59.6%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "/content/AutogluonModels/ag-20250517_022734"
Train Data Rows:    513
Train Data Columns: 128
Label Column:       dG
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (3.43, -23.62, -3.91591, 3.80544)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Pr

[1000]	valid_set's rmse: 1.85799
[2000]	valid_set's rmse: 1.84867
[3000]	valid_set's rmse: 1.84827


	-1.8481	 = Validation score   (-root_mean_squared_error)
	9.78s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 1.89429
[2000]	valid_set's rmse: 1.88419
[3000]	valid_set's rmse: 1.88359
[4000]	valid_set's rmse: 1.88345
[5000]	valid_set's rmse: 1.88342
[6000]	valid_set's rmse: 1.88342
[7000]	valid_set's rmse: 1.88342
[8000]	valid_set's rmse: 1.88342
[9000]	valid_set's rmse: 1.88342
[10000]	valid_set's rmse: 1.88342


	-1.8834	 = Validation score   (-root_mean_squared_error)
	94.64s	 = Training   runtime
	0.36s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-1.9295	 = Validation score   (-root_mean_squared_error)
	9.63s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: CatBoost ...
	-1.7792	 = Validation score   (-root_mean_squared_error)
	578.66s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-2.0687	 = Validation score   (-root_mean_squared_error)
	3.08s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-2.4785	 = Validation score   (-root_mean_squared_error)
	1.19s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ...
	-1.8093	 = Validation score   (-root_mean_squared_error)
	15.44s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-1.3879	 = Validation score   (-root_mean_squared_error)
	68.54s	 = Training   runtime
	0.08s	 = Validation

...

[5000]	valid_set's rmse: 1.77371
[6000]	valid_set's rmse: 1.77371
[7000]	valid_set's rmse: 1.77371
[8000]	valid_set's rmse: 1.77371


(_dystack pid=25005) 	-1.948	 = Validation score   (-root_mean_squared_error)
(_dystack pid=25005) 	1242.13s	 = Training   runtime
(_dystack pid=25005) 	2.41s	 = Validation runtime
(_dystack pid=25005) Fitting model: LightGBM_r96_BAG_L1 ...
(_dystack pid=25005) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.95%)


[9000]	valid_set's rmse: 1.77371
(_ray_fit pid=50711) [1000]	valid_set's rmse: 2.84284
(_ray_fit pid=50711) [2000]	valid_set's rmse: 2.75854
(_ray_fit pid=50711) [3000]	valid_set's rmse: 2.7387
(_ray_fit pid=50711) [4000]	valid_set's rmse: 2.72926
[10000]	valid_set's rmse: 1.77371
(_ray_fit pid=50711) [5000]	valid_set's rmse: 2.72305
(_ray_fit pid=50711) [6000]	valid_set's rmse: 2.7158
(_ray_fit pid=50711) [7000]	valid_set's rmse: 2.70627
(_ray_fit pid=50711) [8000]	valid_set's rmse: 2.70221
(_ray_fit pid=50711) [9000]	valid_set's rmse: 2.703
(_ray_fit pid=50711) [10000]	valid_set's rmse: 2.7091


	-1.7737	 = Validation score   (-root_mean_squared_error)
	414.17s	 = Training   runtime
	0.88s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.882, 'LightGBMLarge': 0.118}
	-1.3801	 = Validation score   (-root_mean_squared_error)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1210.56s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 106.7 rows/s (103 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250517_022734")


{'root_mean_squared_error': np.float64(-1.2733662379475887), 'mean_squared_error': -1.6214615759447955, 'mean_absolute_error': -0.7437502606334381, 'r2': 0.8976392046289446, 'pearsonr': 0.9553439238605232, 'median_absolute_error': np.float64(-0.4189668273925782)}
(_ray_fit pid=50888) [1000]	valid_set's rmse: 2.75893
予測値 (kcal/mol) 上位 5 行:
 86     -1.328596
192    -5.085087
479    -8.995502
616   -18.571823
72      1.394652
Name: dG, dtype: float32


In [ ]:
print(predictor.evaluate(test_df))

{'root_mean_squared_error': np.float64(-1.2733662379475887), 'mean_squared_error': -1.6214615759447955, 'mean_absolute_error': -0.7437502606334381, 'r2': 0.8976392046289446, 'pearsonr': 0.9553439238605232, 'median_absolute_error': np.float64(-0.4189668273925782)}


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/04_tech/molecule_embeddings/freesolv_dimenetpp.csv")
drop_cols = ["id", "smiles"]
data = df.drop(columns=drop_cols, errors="ignore")


train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

predictor = TabularPredictor(label="dG", eval_metric="root_mean_squared_error").fit(
    train_data=train_df,
    # presets="best_quality",
    presets="medium_quality",
    time_limit=None,
)

perf = predictor.evaluate(test_df)  # RMSE, MAE などが表示される
print(perf)

pred = predictor.predict(test_df.drop(columns=["dG"]))
print("予測値 (kcal/mol) 上位 5 行:\n", pred.head())


No path specified. Models will be saved in: "AutogluonModels/ag-20250517_030017"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       8.95 GB / 12.67 GB (70.6%)
Disk Space Avail:   63.79 GB / 107.72 GB (59.2%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "/content/AutogluonModels/ag-20250517_030017"
Train Data Rows:    513
Train Data Columns: 128
Label Column:       dG
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (3.43, -23.62, -3.91591, 3.80544)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Pr

[1000]	valid_set's rmse: 1.80751
[2000]	valid_set's rmse: 1.7866
[3000]	valid_set's rmse: 1.78515
[4000]	valid_set's rmse: 1.78487
[5000]	valid_set's rmse: 1.78491


	-1.7849	 = Validation score   (-root_mean_squared_error)
	16.49s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 1.87945
[2000]	valid_set's rmse: 1.84294
[3000]	valid_set's rmse: 1.83583
[4000]	valid_set's rmse: 1.83423
[5000]	valid_set's rmse: 1.83398
[6000]	valid_set's rmse: 1.83393
[7000]	valid_set's rmse: 1.83391
[8000]	valid_set's rmse: 1.83391
[9000]	valid_set's rmse: 1.83391
[10000]	valid_set's rmse: 1.83391


	-1.8339	 = Validation score   (-root_mean_squared_error)
	90.27s	 = Training   runtime
	0.54s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-2.0745	 = Validation score   (-root_mean_squared_error)
	11.32s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: CatBoost ...
(_dystack pid=4887) 	-2.0249	 = Validation score   (-root_mean_squared_error)
(_dystack pid=4887) 	776.18s	 = Training   runtime
(_dystack pid=4887) 	5.78s	 = Validation runtime
(_dystack pid=4887) Fitting model: ExtraTrees_r172_BAG_L1 ...
(_dystack pid=4887) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
(_dystack pid=4887)   warnings.warn(
(_dystack pid=4887) 	-2.4022	 = Validation score   (-root_mean_squared_error)
(_dystack pid=4887) 	15.36s	 = Training   

{'root_mean_squared_error': np.float64(-1.132695834402747), 'mean_squared_error': -1.2829998532733349, 'mean_absolute_error': -0.711876799709113, 'r2': 0.919005860274251, 'pearsonr': 0.9613423538407296, 'median_absolute_error': np.float64(-0.4106175994873047)}
予測値 (kcal/mol) 上位 5 行:
 86     -1.197372
192    -5.232316
479    -9.889773
616   -20.507742
72      1.660337
Name: dG, dtype: float32


In [ ]:
print(predictor.evaluate(test_df))

{'root_mean_squared_error': np.float64(-1.132695834402747), 'mean_squared_error': -1.2829998532733349, 'mean_absolute_error': -0.711876799709113, 'r2': 0.919005860274251, 'pearsonr': 0.9613423538407296, 'median_absolute_error': np.float64(-0.4106175994873047)}
